In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))
train_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,57107,57107,13375.0,0.1797,483.34,MORTGAGE,223000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,577150.0,122018.0,32000.0,170200.0,N,N
1,141451,141451,21000.0,0.1308,478.68,MORTGAGE,123000.0,Source Verified,low_risk,n,...,85.0,33.3,0.0,0.0,132750.0,27896.0,15900.0,35398.0,N,N
2,321143,321143,20000.0,0.1240,448.95,MORTGAGE,197000.0,Source Verified,low_risk,n,...,85.7,33.3,0.0,0.0,628160.0,114043.0,22600.0,90340.0,N,N
3,11778,11778,3000.0,0.1240,100.22,RENT,45000.0,Not Verified,low_risk,n,...,100.0,16.7,1.0,0.0,42006.0,20761.0,19900.0,15406.0,N,N
4,169382,169382,30000.0,0.1612,1056.49,MORTGAGE,133000.0,Source Verified,low_risk,n,...,100.0,66.7,0.0,0.0,283248.0,109056.0,79500.0,58778.0,N,N


In [3]:
test_df.head()

,Unnamed: 0,index,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,67991,67991,40000.0,0.0819,814.70,MORTGAGE,140000.0,Not Verified,low_risk,n,...,97.7,0.0,0.0,0.0,527975.0,70914.0,74600.0,99475.0,N,N
1,25429,25429,6000.0,0.1524,208.70,RENT,55000.0,Not Verified,low_risk,n,...,66.7,0.0,0.0,0.0,34628.0,23460.0,5900.0,23628.0,N,N
2,38496,38496,3600.0,0.1695,128.27,RENT,42000.0,Not Verified,low_risk,n,...,100.0,0.0,0.0,0.0,23100.0,19183.0,7300.0,15000.0,N,N
3,19667,19667,20000.0,0.1524,478.33,RENT,100000.0,Not Verified,low_risk,n,...,100.0,50.0,0.0,0.0,56481.0,43817.0,13800.0,35981.0,N,N
4,37505,37505,3600.0,0.1240,120.27,RENT,50000.0,Not Verified,low_risk,n,...,100.0,25.0,0.0,0.0,45977.0,32448.0,21000.0,24977.0,N,N


In [4]:
del train_df['Unnamed: 0']

In [5]:
# Convert categorical data to numeric and separate target feature for training data
X_train=train_df.drop(['loan_status'], axis = 1)
X_train_df=pd.get_dummies(X_train)
Y_train=LabelEncoder().fit_transform(train_df['loan_status'])
Y_train

array([1, 1, 1, ..., 0, 0, 0])

In [6]:
#Check if there are any missing varibles in the training data set
test_df.isnull().sum(axis=0)

Unnamed: 0                    0
index                         0
loan_amnt                     0
int_rate                      0
installment                   0
                             ..
total_bal_ex_mort             0
total_bc_limit                0
total_il_high_credit_limit    0
hardship_flag                 0
debt_settlement_flag          0
Length: 86, dtype: int64

In [7]:
del test_df['Unnamed: 0']

In [8]:
# Convert categorical data to numeric and separate target feature for testing data
X_test=test_df.drop(['loan_status'], axis=1)
X_test=pd.get_dummies(X_test)
Y_test= LabelEncoder().fit_transform(test_df['loan_status'])
Y_test

array([1, 1, 1, ..., 0, 0, 0])

In [9]:
# add missing dummy variables to testing set
test_df.isnull().sum(axis=0)

index                         0
loan_amnt                     0
int_rate                      0
installment                   0
home_ownership                0
                             ..
total_bal_ex_mort             0
total_bc_limit                0
total_il_high_credit_limit    0
hardship_flag                 0
debt_settlement_flag          0
Length: 85, dtype: int64

In [10]:
X_test['verification_status_Verified']=0
X_test.head()

,index,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Source Verified,verification_status_Verified,pymnt_plan_n,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,hardship_flag_N,hardship_flag_Y,debt_settlement_flag_N
0,67991,40000.0,0.0819,814.70,140000.0,19.75,0.0,1.0,18.0,0.0,...,0,0,1,0,1,1,0,1,0,1
1,25429,6000.0,0.1524,208.70,55000.0,11.52,2.0,0.0,8.0,0.0,...,0,0,1,0,1,1,0,1,0,1
2,38496,3600.0,0.1695,128.27,42000.0,6.74,0.0,0.0,6.0,0.0,...,0,0,1,0,1,1,0,1,0,1
3,19667,20000.0,0.1524,478.33,100000.0,12.13,0.0,2.0,7.0,0.0,...,0,0,1,0,1,1,0,1,0,1
4,37505,3600.0,0.1240,120.27,50000.0,16.08,0.0,3.0,6.0,0.0,...,0,0,1,0,1,1,0,1,0,1


# Prediction Model

In [15]:
# Train the Logistic Regression model on the unscaled data and print the model score
model=LogisticRegression(max_iter= 15000)
model.fit(X_train,Y_train)

print(f"Training Data Score:{model.score(X_train,Y_train)}")
print(f"Testing Data Score: {model.score(X_test,Y_test)}")

ValueError: could not convert string to float: 'MORTGAGE'

In [16]:
# Train a Random Forest Classifier model and print the model score
model=RandomForestClassifier(random_state=1,n_estimators=50).fit(X_train,Y_train)
model.fit(X_train,Y_train)
print(f"Training Data Score:{model.score(X_train,Y_train)}")
print(f"Testing Data Score:{model.score(X_test,Y_test)}")

ValueError: could not convert string to float: 'MORTGAGE'

In [ ]:
# Scale the data
scaler=StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [ ]:
# Train the Logistic Regression model on the scaled data and print the model score
model=LogisticRegression(max_iter=10000).fit(X_train_scaled,y_train)
print(f'Training Score: {model.score(X_train_scaled,Y_train)}')
print(f'Testing Score: {model.score(X_test_scaled,Y_test)}')

In [ ]:
# Train a Random Forest Classifier model on the scaled data and print the model score
model=RandomForestClassifier(random_state=1,n_estimators=50).fit(X_train_scaled,Y_train)
model.fit(X_train_scaled,Y_train)
print(f'Training Score: {model.score(X_train_scaled,Y_train)}')
print(f'Testing Score: {model.score(X_test_scaled,Y_test)}')

# FINAL OUTCOME